<a href="https://www.kaggle.com/code/asmahachaichi/loan-approval-prediction-automl?scriptVersionId=203325527" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install h2o

Note: you may need to restart the kernel to use updated packages.


In [2]:
import h2o
from h2o.automl import H2OAutoML
import numpy as np
from scipy import stats

In [3]:
# Start the H2O cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu320.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu320.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpg5qon8k1
  JVM stdout: /tmp/tmpg5qon8k1/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpg5qon8k1/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 25 days
H2O_cluster_name:,H2O_from_python_unknownUser_bsfyxf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.500 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
train = h2o.import_file("/kaggle/input/playground-series-s4e10/train.csv")
test = h2o.import_file("/kaggle/input/playground-series-s4e10/test.csv")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.head()

id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,37,35000,RENT,0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8,PERSONAL,A,6000,8.9,0.21,N,10,0
3,30,70000,RENT,14,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2,MEDICAL,A,6000,6.92,0.1,N,3,0
5,27,45000,RENT,2,VENTURE,A,9000,8.94,0.2,N,5,0
6,25,45000,MORTGAGE,9,EDUCATION,A,12000,6.54,0.27,N,3,0
7,21,20000,RENT,0,PERSONAL,C,2500,13.49,0.13,Y,3,0
8,37,69600,RENT,11,EDUCATION,D,5000,14.84,0.07,Y,11,0
9,35,110000,MORTGAGE,0,DEBTCONSOLIDATION,C,15000,12.98,0.14,Y,6,0


In [6]:
test.head()

id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
58645,23,69000,RENT,3,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
58646,26,96000,MORTGAGE,6,PERSONAL,C,10000,12.68,0.1,Y,4
58647,26,30000,RENT,5,VENTURE,E,4000,17.19,0.13,Y,2
58648,33,50000,RENT,4,DEBTCONSOLIDATION,A,7000,8.9,0.14,N,7
58649,26,102000,MORTGAGE,8,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4
58650,23,66000,RENT,5,EDUCATION,D,22000,14.09,0.33,N,2
58651,26,75000,OWN,10,PERSONAL,B,8000,10.62,0.11,N,4
58652,23,55000,MORTGAGE,6,PERSONAL,A,6250,6.76,0.12,N,2
58653,32,29124,RENT,0,PERSONAL,C,7200,13.11,0.26,Y,6
58654,22,90000,RENT,4,DEBTCONSOLIDATION,C,10000,13.49,0.11,Y,3


In [7]:
# Convert H2OFrame to Pandas for outlier detection (or you can work directly with H2OFrames)
df = train.as_data_frame()

# Remove rows with Z-score greater than 3 (i.e., extreme outliers)
df_clean = df[(np.abs(stats.zscore(df.select_dtypes(include=[np.number]))) < 3).all(axis=1)]

# Convert cleaned Pandas DataFrame back to H2OFrame
train = h2o.H2OFrame(df_clean)

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
# Specify target and predictors
y = "loan_status"
x = train.columns
x.remove(y)

In [9]:
train[y] = train[y].asfactor()

In [10]:
# Calculate class distribution
class_distribution = train[y].table()
class_distribution_df = class_distribution.as_data_frame()
print(class_distribution)

/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


  loan_status    Count
            0    48175
            1     7635
[2 rows x 2 columns]



In [11]:
# Assign class weights using H2O's ifelse function
train['class_weights'] = train['loan_status'].ifelse(1.0, 0.2)


In [12]:
# train h2o model with a max of 10 models
include_algos = ["XGBoost", "GBM"]
aml = H2OAutoML(max_models=10, seed=1, include_algos=include_algos)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_3_AutoML_1_20241025_113534


Model Summary: 
    number_of_trees
--  -----------------
    77

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.033215346250040606
RMSE: 0.18225077846209767
LogLoss: 0.1239656073733682
Mean Per-Class Error: 0.10912896355481969
AUC: 0.9725614870745539
AUCPR: 0.9095257746152415
Gini: 0.9451229741491078

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3309985163311164
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      47409  766   0.0159   (766.0/48175.0)
1      1545   6090  0.2024   (1545.0/7635.0)
Total  48954  6856  0.0414   (2311.0/55810.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.330999     0.840522  185
max f2                       0.173682     0.840147  239
max f0point5                 0.580646     0.892771  128
max accuracy                 0.449408     0.959255  154
max precision                0.998804     1         0
max recall                   0.0027923    1         393
max specificity              0.998804     1         0
max absolute_mcc             0.38834      0.81855   169
max min_per_class_accuracy   0.107661     0.909497  274
max mean_per_class_accuracy  0.142783     0.911404  254
max tns                      0.998804     48175     0
max fns                      0.998804     7531      0
max fps                      0.000455618  48175     399
max tps                      0.0027923    7635      393
max tnr                      0.998804     1         0
max fnr                      0.998804     0.986379  0
max fpr                      0.000455618  1         399
max tpr                      0.0027923    1         393

Gains/Lift Table: Avg response rate: 13.68 %, avg score: 13.71 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100161                   0.993478           7.30976     7.30976            1                0.996469    1                           0.996469            0.0732155       0.0732155                  630.976   630.976            0.0732155
2        0.0200143                   0.986352           7.29666     7.30321            0.998208         0.989909    0.999105                    0.993192            0.0729535       0.146169                   629.666   630.321            0.146148
3        0.0300125                   0.979308           7.27046     7.2923             0.994624         0.982833    0.997612                    0.989741            0.0726916       0.218861                   627.046   629.23             0.218777
4        0.0400108                   0.970656           7.19186     7.2672             0.983871         0.975296    0.994178                    0.986131            0.0719057       0.290766                   619.186   626.72             0.290496
5        0.050009                    0.960395           7.25736     7.26523            0.992832         0.965622    0.993909                    0.982031            0.0725606       0.363327                   625.736   626.523            0.362974
6        0.1                         0.68309            6.74122     7.00327            0.922222         0.886987    0.958072                    0.934517            0.337001        0.700327                   574.122   600.327            0.69547
7        0.150009                    0.195479           3.

In [13]:
#Predict target from test data
pred_h2o = aml.predict(test)

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [14]:
pred_h2o.head()

predict,p0,p1
1,0.00202578,0.997974
0,0.980872,0.0191276
1,0.282976,0.717024
0,0.971999,0.028001
0,0.936325,0.063675
1,0.0263852,0.973615
0,0.998172,0.00182803
0,0.991401,0.00859927
1,0.606878,0.393122
0,0.966783,0.0332172


In [15]:
import pandas as pd

# Convert the H2O frame to a Pandas DataFrame
df = pred_h2o.as_data_frame()
test_df = test.as_data_frame()

# Calculate loan_status by taking the argmax of p0 and p1
df['loan_status'] = df[['p0', 'p1']].idxmax(axis=1).apply(lambda x: int(x[-1]))
result_df = pd.concat([test_df[['id']], df[['loan_status']]], axis=1)

# Save the DataFrame to a CSV file
result_df.to_csv('/kaggle/working/submission.csv', index=False)


/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
/opt/conda/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"
